## 1. Initialization

In [8]:
import sys
import os

os.chdir("../")
path1 = os.getcwd()

os.chdir("coppeliasim_utils")
path2 = os.getcwd()


sys.path.append(path1)
sys.path.append(path2)


from indy_utils import indydcp_client as client
from coppeliasim_utils import sim

from time import sleep
import time
import json
import threading
import numpy as np
import math
import keyboard
import threading

robot_ip = "192.168.0.15"  # Robot (Indy) IP
robot_name = "NRMK-Indy7"  # Robot name (Indy7)

# Create class object (real robot)
indy_actual = client.IndyDCPClient(robot_ip, robot_name)

# Crate class object (simulation robot)
indy_virtual = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)

# Actual robot (Created: True, Not created: False)
# Virtual robot (Created: True, Not created: False)
print("real robot: {0}\nsimulation robot: {1}".format(bool(indy_actual), bool(not indy_virtual)))

real robot: True
simulation robot: True


## 2. Communication reset

In [7]:
# Reset TCP-IP communication (real robot)
# For real robot
#indy_actual.connect()
indy_actual.disconnect()

# For simulation robot
sim.simxFinish(indy_virtual)

## 3. Position data acquisition

In [9]:
# For real robot
indy_actual.connect()
j_pos_act = indy_actual.get_joint_pos()
t_pos_act = indy_actual.get_task_pos()

# For simulation robot
handles = []
j_pos_vir = []
t_pos_vir = []

for i in range(6):
    object_name = 'joint' + str(i)
    result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
    if result != sim.simx_return_ok:
        raise Exception('could not get object handle for first joint')                
    else:
        handles.append(handle)
        result, jnt_angle = sim.simxGetJointPosition(indy_virtual, handle, sim.simx_opmode_oneshot)
        j_pos_vir.append(jnt_angle/math.pi*180)
        
object_name = 'tool_coordinate'
result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)

result, obj_pos = sim.simxGetObjectPosition(indy_virtual, handle, -1, sim.simx_opmode_oneshot)
result, obj_rot = sim.simxGetObjectOrientation(indy_virtual, handle, -1, sim.simx_opmode_oneshot)

print()

for i in range(3):
    t_pos_vir.append(obj_pos[i])

for i in range(3):
    t_pos_vir.append(obj_rot[i]/np.pi*180)
        
print("Joint Position (real):", np.round(j_pos_act, 2))
print("Joint Position (simulation):", np.round(j_pos_vir, 2))

print()
print("Task Position (real):", np.round(t_pos_act,2))
print("Task Position (simulation):", np.round(t_pos_vir,2))


Connect: Server IP (192.168.0.15)

Joint Position (real): [-2.900e-01 -5.038e+01 -2.980e+01  5.000e-02 -9.996e+01 -2.900e-01]
Joint Position (simulation): [0. 0. 0. 0. 0. 0.]

Task Position (real): [ 6.9000e-01 -1.9000e-01  4.2000e-01 -5.0000e-02 -1.7985e+02  1.0000e-02]
Task Position (simulation): [0. 0. 0. 0. 0. 0.]


## 4. Set position

In [10]:
### For actual robot
indy_actual.connect()

# Zero position
#j_pos_act = [0, 0, 0, 0, 0, 0]

# Home position
j_pos_act = [0, -15, -90, 0, -75, 0]
indy_actual.joint_move_to(j_pos_act)  # Move 6th joint

indy_actual.disconnect()

### For virtual robot
handles = []
#Zero position
#j_pos_vir = [0, 0, 0, 0, 0, 0]

# Home position
j_pos_vir = [0, -15, -90, 0, -75, 0]

for i in range(6):
    object_name = 'joint' + str(i)
    #print(object_name)
    result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
    if result != sim.simx_return_ok:
        raise Exception('could not get object handle for first joint')                
    else:
        handles.append(handle)
        sim.simxSetJointTargetPosition(indy_virtual, handle, j_pos_vir[i]*math.pi/180, sim.simx_opmode_oneshot)

Connect: Server IP (192.168.0.15)


## 5. Synchronization

### 5-2. Synchronization (w threaded)

In [11]:
def cps_program():
    indy_actual.connect()

    handles = []

    for i in range(6):
            object_name = 'joint' + str(i)    
            #print(object_name)
            result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
            handles.append(handle)

    print(handles)

    isCtrOn = True

    while isCtrOn:
        j_pos = indy_actual.get_joint_pos()
        time.sleep(0.002)

        for i in range(6):
            sim.simxSetJointTargetPosition(indy_virtual, handles[i] , float(j_pos[i])*math.pi/180 ,sim.simx_opmode_oneshot)

        if keyboard.is_pressed("q"):
            isCtrOn = False
            print("Escape the program")

    indy_actual.disconnect()

t = threading.Thread(target=cps_program)
t.start()
print("Main Thread")

Main Thread
Connect: Server IP (192.168.0.15)
[25, 27, 29, 31, 33, 35]


### 5-3. Motion test

In [6]:
indy_actual.go_home()

In [ ]:
indy_actual.stop_motion()

In [5]:
indy_actual.go_zero()

## 6. 로봇 동작 구현

In [ ]:
import numpy as np
from indy_utils import indydcp_client as client

 
center = [0.51, -0.19, 0.42]  #원의 중심
radius =0.18   #원 반지름
#step_size = 0.628
step_size = 1.5708 # 90도 각도 스탭


trajectory = []
angle = 0
while angle < 2*np.pi:  # Do a full circle
    x = center[0] + radius*np.cos(angle)
    y = center[1] + radius*np.sin(angle)
    z = center[2]
    point = [x, y, z,0,180,0]  # 점 찍기
    trajectory.append(point)
    angle += step_size

    
trajectory.append(trajectory[0])        
# Connect to actual robot
#indy_actual.connect()
level = 7
level2 = 3
# Move the robot along the trajectory
indy_actual.set_task_vel_level(level)
indy_actual.set_collision_level(level2)


def check_robot_status():
    while True:
        result, temp = sim.simxGetIntegerSignal(indy_virtual, 'Prx_sensor', sim.simx_opmode_oneshot)
        # print(temp)
        status = indy_actual.get_robot_status()
        print(status)
        
        if status.get('collision') == 0 :
            if status.get('busy') == 1:  # 로봇 바쁜 상태
                #print("busy = 1")
                time.sleep(0.5)
                # threading.Event().wait(0.5)
            else:  # Robot has stopped moving
                #print("busy = 0")  # Move to the next coordinate
                time.sleep(0.5)
                break
        else:
            time.sleep(5)
            indy_actual.reset_robot()
            


#count = 0
while True:
    result, temp = sim.simxGetIntegerSignal(indy_virtual, 'Prx_sensor', sim.simx_opmode_oneshot)
    print(temp)
    #if temp == 0:
    for point in trajectory:
            #indy_actual.task_move_to(point)
            result, temp = sim.simxGetIntegerSignal(indy_virtual, 'Prx_sensor', sim.simx_opmode_oneshot)
            print(temp)
            if temp ==0:
                indy_actual.task_move_to(point)
            elif temp ==1:
                check_robot_status()
                indy_actual.go_home()
                check_robot_status()
                indy_actual.task_move_to([0.45, -0.19, 0.5, 0, 180, 0]) #위로 들어올리기
                check_robot_status()
                indy_actual.task_move_to([0.05, -0.6, 0.5, 0, 180, 0]) #그릇까지 이동
                check_robot_status()
                indy_actual.task_move_to([0.015, -0.6, 0.5, 0, 165, 0])  #국 따르기
                check_robot_status()
                indy_actual.task_move_to([0.05, -0.6, 0.5, 0, 180, 0]) #각도 수정
                check_robot_status()
                indy_actual.task_move_to([0.45, -0.19, 0.5, 0, 180, 0]) #Z home
                check_robot_status()
            #print(count)
            #count +=1
                #indy_actual.go_home()
            while True:
                result, temp = sim.simxGetIntegerSignal(indy_virtual, 'Prx_sensor', sim.simx_opmode_oneshot)
                # print(temp)
                status = indy_actual.get_robot_status()
                print(status)

                if status.get('collision') == 0 :
                    if status.get('busy') == 1:  # 로봇 바쁜 상태
                        #print("busy = 1")
                        time.sleep(0.5)
                        # threading.Event().wait(0.5)
                    else:  # Robot has stopped moving
                        #print("busy = 0")  # Move to the next coordinate
                        time.sleep(0.5)
                        break
                else:
                    time.sleep(5)
                    indy_actual.reset_robot()

        
    
    
    
    
# Disconnect from actual robot
#indy_actual.disconnect()

0
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, '

{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'erro

{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'er

0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 1, 'error': 0, 'busy': 0, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 0, 'emergency': 0, 'collision': 1, 'er

{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'er

...

0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0,

{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error'

0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'er

{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
1
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'erro

{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 1, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
0
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 1, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
{'ready': 1, 'emergency': 0, 'collision': 0, '